## AlphaTims

In [30]:
import os
import re
import numpy as np
import pandas as pd
import panel as pn

import holoviews as hv
from holoviews import opts

# pn.extension('bokeh')


In [31]:
css = '''
.bk.opt {
    position: relative;
    display: block;
    left: 75px;
    top: 0px;
    width: 80px;
    height: 80px;
}

h1 {
    color: #045082;
    font-size: 45px;
    line-height: 0.6;
    text-align: center;
}

h2 {
    color: #045082;
    text-align: center;
}

.bk.main-part {
    background-color: #EAEAEA;
    font-size: 17px;
    line-height: 23px;
    letter-spacing: 0px;
    font-weight: 500;
    color: #045082;
    text-align: center;
    position: relative !important;
    margin-left: auto;
    margin-right: auto;
    width: 40%;
}

.bk-root .bk-btn-primary {
    background-color: #045082;
    font-size: 11px;
    font-weight: 700;
    text-transform: uppercase;
    letter-spacing: 1.5px;
}

.bk.alert-danger {
    background-color: #EAEAEA;
    color: #c72d3b;
    border: 0px #EAEAEA solid;
    padding: 0;
}

.settings {
    background-color: #EAEAEA;
    border: 2px solid #045082;
}

'''
pn.extension(raw_css=[css])

In [32]:
### LOCAL VARIABLES

DATASET = None

In [33]:
### HEADER

header_titel = pn.pane.Markdown(
    '# AlphaTims', 
    width=1250
)
mpi_biochem_logo = pn.pane.PNG(
    '../alphatims/img/mpi_logo.png', 
    link_url='https://www.biochem.mpg.de/en', 
    width=60, 
    height=60,
    align='start'
)
mpi_logo = pn.pane.JPG(
    '../alphatims/img/max-planck-gesellschaft.jpg', 
    link_url='https://www.biochem.mpg.de/en',
    height=62,
    embed=True,
    width=62,
    margin=(5, 0, 0, 5),
    css_classes=['opt']
)
github_logo = pn.pane.PNG(
    '../alphatims/img/github.png',
    link_url='https://github.com/MannLabs/alphatims',
    height=70,
)

header = pn.Row(
    mpi_biochem_logo,
    mpi_logo,
    header_titel,
    github_logo,
    height=73
)

In [34]:
### MAIN PART

project_description = pn.pane.Markdown(
    """### AlphaTIMS is an open-source Python package for fast accessing Bruker TimsTOF data. It provides a very efficient indexed data structure that allows to access four-dimensional TIMS-time of flight data in the standard numerical python (NumPy) manner. AlphaTIMS is a key enabling tool to deal with the large and high dimensional TIMS data.""",
    margin=(10, 0, 0, 0), 
    css_classes=['main-part'], 
    width=615
)

divider_descr = pn.pane.HTML(
    '<hr style="height: 6px; border:none; background-color: #045082; width: 1010px">', 
    width=1510, 
    align='center'
)

upload_file = pn.widgets.TextInput(
    name='Specify an experimental file:',
    placeholder='Enter the whole path to Bruker .d folder or .hdf file',
    width=800,
    margin=(15,15,0,15) 
)

upload_button = pn.widgets.Button(
    name='Upload  Data', 
    button_type='primary', 
    height=31,
    width=100,
    margin=(34,20,0,20)
)

upload_spinner = pn.indicators.LoadingSpinner(
    value=False,
    bgcolor='light',
    color='secondary',
    margin=(30,20,0,10),
    width=40, 
    height=40
)

upload_error = pn.pane.Alert(
    width=400,
    alert_type="danger", 
    margin=(-15,0,10,260),
)

main_part = pn.Column(
    project_description,
    divider_descr,
    pn.Row(
        upload_file,
        upload_button,
        upload_spinner,
        align='center'
    ),
    upload_error,
    background='#eaeaea',
    width=1510,
    height=350,
    margin=(5, 0, 10, 0)
)

In [35]:
### SETTINGS

settings_title = pn.pane.Markdown(
    '## Parameters',
    align='center', 
    margin=(10,0,0,0)
)
settings_divider = pn.pane.HTML(
    '<hr style="height: 3.5px; border:none; background-color: #045082; width: 420px;">', 
    align='center',
    margin=(0, 10, 0, 20)
)
frame_slider = pn.widgets.IntRangeSlider(
    name='Frames',
    start=1,
    step=1,
    margin=(0, 20)
)

scan_slider = pn.widgets.IntRangeSlider(
    name='Scans',
    start=1,
    step=1,
    margin=(5, 20)
)

quad_slider = pn.widgets.RangeSlider(
    name='Quad',
    start=-1,
    value=(-1, -1),
    step=1,
    margin=(5, 20)
)

tof_slider = pn.widgets.IntRangeSlider(
    name='TOF',
    start=1,
    step=1,
    margin=(5, 20)
)

settings = pn.layout.WidgetBox(
    settings_title,
    settings_divider,
    frame_slider,
    scan_slider,
    quad_slider,
    tof_slider,
    width=460,
    align='start',
    margin=(0,0,0,0),
    css_classes=['settings']
)

In [36]:
### plotting options
chrom_opts = opts.Curve(
    title="Chromatogram",
    width=1000, 
    height=300, 
    xlabel='RT, min', 
    ylabel='Intensity',
    line_width=1, 
    yformatter='%.1e', 
    shared_axes=True
)

In [43]:
### FUNCTIONS
# preload data
@pn.depends(
    upload_button.param.clicks,
    watch=True
)
def upload_data(_):
    import sys
    sys.path.append('../')
    import alphatims.bruker
    global DATASET
    if upload_file.value.endswith(".d") or upload_file.value.endswith(".hdf"):
        upload_error.object = None
        if not DATASET or DATASET.bruker_d_folder_name != upload_file.value:
            upload_spinner.value = True
            DATASET = alphatims.bruker.TimsTOF(
                upload_file.value
            )
            upload_spinner.value = False
    else:
        upload_error.object = '#### Please, specify a path to .d Bruker folder or .hdf file.'

### PLOTTING
def visualize_chrom():
    if DATASET:
        data = DATASET.frames.query('MsMsType == 0')[['Time', 'SummedIntensities']]
        data['RT'] = data['Time'] / 60
        chrom = hv.Curve(
            data=data, 
            kdims=['RT', 'SummedIntensities']
        ).opts(
            chrom_opts
        )
        return chrom
        
### SHOW SETTINGS/PLOTS
@pn.depends(
    upload_button.param.clicks
)        
def show_settings(_):
    if DATASET:
        frame_slider.end = DATASET.frame_max_index + 1
        scan_slider.end = DATASET.scan_max_index + 1
        scan_slider.value = (1, DATASET.scan_max_index + 1)
        quad_slider.end = DATASET.quad_max_index
        tof_slider.end = DATASET.tof_max_index + 1
        tof_slider.value = (1, DATASET.tof_max_index + 1)
        return settings
    else:
        return None
    
    
    
    
@pn.depends(
    frame_slider.param.value,
    scan_slider.param.value,
    quad_slider.param.value,
    tof_slider.param.value,
    watch=True
)
def show_plots(
    frame_values,
    scan_values,
    quad_values,
    tof_values,
):
    layout_plots = pn.Column(
        visualize_chrom()
    )  
    return layout_plots

In [ ]:
layout = pn.Column(
    header,
    main_part, 
    pn.Row(
        show_settings,
        show_plots,
    ),
)
layout.servable()